In [ ]:
from gtts import gTTS
import os

text = "شو هي الأسماء اللي علمها الله سبحانه وتعالى للنبي آدم؟"

# Create TTS object
tts = gTTS(text=text, lang="ar")

tts.save("output.mp3")

os.system("start output.mp3")

0

In [ ]:
from dotenv import load_dotenv
from elevenlabs.client import ElevenLabs
from elevenlabs import play
import os

load_dotenv()

elevenlabs = ElevenLabs(
  api_key=os.getenv("ELEVENLABS_API_KEY"),
)

audio = elevenlabs.text_to_speech.convert(
    text="شو هي الأسماء اللي علمها الله سبحانه وتعالى للنبي آدم؟",
    voice_id="JBFqnCBsd6RMkjVDRZzb",
    model_id="eleven_multilingual_v2",
    output_format="mp3_44100_128",
)

from pathlib import Path

# Convert generator to bytes
audio_bytes = b"".join(audio)

with open("output2.mp3", "wb") as f:
    f.write(audio_bytes)

print("Audio saved as output2.mp3")


Audio saved as output2.mp3


ModuleNotFoundError: No module named 'playsound'

In [1]:
# example.py
import os
import time
import base64
from contextlib import ExitStack
from io import BytesIO
from dotenv import load_dotenv
from elevenlabs.client import ElevenLabs

load_dotenv()

elevenlabs = ElevenLabs(
  api_key=os.getenv("ELEVENLABS_API_KEY"),
)

voice = elevenlabs.voices.pvc.create(
    name="Al-Sayed Voice Clone",
    language="ar",
    description="A professional voice clone of Al-Sayed Hashem voice"
)

print(voice)


voice_id='0kMo1XiwRkzMPV4kNcGU'


In [2]:
print(voice.voice_id)

0kMo1XiwRkzMPV4kNcGU


In [5]:
# Define the list of file paths explicitly
# Replace with the paths to your audio and/or video files.
# The more files you add, the better the clone will be.
sample_file_paths = [
    r"C:\Users\Lenovo\Downloads\project\sayed_clips\sayed_01.wav"
]

samples = None

files_to_upload = []
# Use ExitStack to manage multiple open files
with ExitStack() as stack:
    for filepath in sample_file_paths:
        # Open each file and add it to the stack
        audio_file = stack.enter_context(open(filepath, "rb"))
        filename = os.path.basename(filepath)

        # Create a File object for the SDK
        files_to_upload.append(
            BytesIO(audio_file.read())
        )

    samples = elevenlabs.voices.pvc.samples.create(
        voice_id=voice.voice_id,
        files=files_to_upload # Pass the list of File objects
    )


In [6]:
sample_ids_to_check = []
for sample in samples:
    if sample.sample_id:
        print(f"Starting separation for sample: {sample.sample_id}")
        elevenlabs.voices.pvc.samples.speakers.separate(
            voice_id=voice.voice_id,
            sample_id=sample.sample_id
        )
        sample_ids_to_check.append(sample.sample_id)

while sample_ids_to_check:
    # Create a copy of the list to iterate over, so we can remove items from the original
    ids_in_batch = list(sample_ids_to_check)
    for sample_id in ids_in_batch:
        status_response = elevenlabs.voices.pvc.samples.speakers.get(
            voice_id=voice.voice_id,
            sample_id=sample_id
        )
        status = status_response.status
        print(f"Sample {sample_id} status: {status}")
        if status == "completed" or status == "failed":
            sample_ids_to_check.remove(sample_id)

    if sample_ids_to_check:
        # Wait before the next poll cycle
        time.sleep(5) # Wait for 5 seconds

print("All samples have been processed or removed from polling.")


Starting separation for sample: tdhQuWNmQ0tMIX3jfDNj
Sample tdhQuWNmQ0tMIX3jfDNj status: pending
Sample tdhQuWNmQ0tMIX3jfDNj status: completed
All samples have been processed or removed from polling.


In [9]:
# Get the list of samples from the voice created in Step 3
voice = elevenlabs.voices.get(voice_id="0kMo1XiwRkzMPV4kNcGU")

samples = voice.samples

# Loop over each sample and save the audio for each speaker to a file
speaker_audio_output_dir = "path/to/speakers/"
if not os.path.exists(speaker_audio_output_dir):
    os.makedirs(speaker_audio_output_dir)

for sample in samples:
    speaker_info = elevenlabs.voices.pvc.samples.speakers.get(
        voice_id=voice.voice_id,
        sample_id=sample.sample_id
    )

    # Proceed only if separation is actually complete
    if getattr(speaker_info, 'status', 'unknown') != "completed":
        continue

    if hasattr(speaker_info, 'speakers') and speaker_info.speakers:
        speaker_list = speaker_info.speakers
        if isinstance(speaker_info.speakers, dict):
            speaker_list = speaker_info.speakers.values()

        for speaker in speaker_list:
            audio_response = elevenlabs.voices.pvc.samples.speakers.audio.get(
                voice_id=voice.voice_id,
                sample_id=sample.sample_id,
                speaker_id=speaker.speaker_id
            )

            audio_base64 = audio_response.audio_base_64
            audio_data = base64.b64decode(audio_base64)
            output_filename = os.path.join(speaker_audio_output_dir, f"sample_{sample.sample_id}_speaker_{speaker.speaker_id}.mp3")

            with open(output_filename, "wb") as f:
                f.write(audio_data)


In [10]:
elevenlabs.voices.pvc.samples.update(
    voice_id=voice.voice_id,
    sample_id=sample.sample_id,
    selected_speaker_ids=[speaker.speaker_id]
)

ApiError: headers: {'date': 'Wed, 03 Sep 2025 07:57:37 GMT', 'server': 'uvicorn', 'content-length': '130', 'content-type': 'application/json', 'access-control-allow-origin': '*', 'access-control-allow-headers': '*', 'access-control-allow-methods': 'POST, PATCH, OPTIONS, DELETE, GET, PUT', 'access-control-max-age': '600', 'strict-transport-security': 'max-age=31536000; includeSubDomains', 'x-trace-id': 'b3f44a7e4ea252b12228dbe9ada48f28', 'x-region': 'us-central1', 'via': '1.1 google, 1.1 google', 'alt-svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'}, status_code: 400, body: {'detail': {'status': 'speaker_separation_incomplete', 'message': 'Speaker separation must be completed before selecting a speaker.'}}

## AudioStack

In [5]:
import requests

url = "https://v2.api.audio/speech/voice-cloning"
headers = {
    "Content-Type": "application/json",
    "Authorization": "COG_Developers d54979fe-d8e9-498a-88e2-caf7b0db081a"
}
payload = {
    "fileIds": [r"/Users/mohamad/Documents/GitHub/Personalized-RAG-Chatbot/voices/lv_0_20250912110033 1.mp3",
                r"/Users/mohamad/Documents/GitHub/Personalized-RAG-Chatbot/voices/WhatsApp Audio 2025-08-23 at 4.00.19 PM.mpeg",
                r"/Users/mohamad/Documents/GitHub/Personalized-RAG-Chatbot/voices/WhatsApp Audio 2025-08-23 at 12.13.58 PM.mp4",
                r"/Users/mohamad/Documents/GitHub/Personalized-RAG-Chatbot/voices/WhatsApp Audio 2025-08-31 at 9.05.39 AM.mpeg",
                r"/Users/mohamad/Documents/GitHub/Personalized-RAG-Chatbot/voices/WhatsApp Audio 2025-08-31 at 9.08.13 AM.mpeg",
                r"/Users/mohamad/Documents/GitHub/Personalized-RAG-Chatbot/voices/WhatsApp Audio 2025-08-31 at 9.10.01 AM.mpeg",
                r"/Users/mohamad/Documents/GitHub/Personalized-RAG-Chatbot/voices/WhatsApp Audio 2025-08-31 at 9.13.15 AM.mpeg",
                r"/Users/mohamad/Documents/GitHub/Personalized-RAG-Chatbot/voices/WhatsApp Audio 2025-08-31 at 9.18.35 AM.mpeg",
                r"/Users/mohamad/Documents/GitHub/Personalized-RAG-Chatbot/voices/WhatsApp Audio 2025-09-12 at 6.13.44 PM.mp4"],
    "alias": "pro_clone",
    "engine": "voice_engine_3",
    "metadata": {
        "gender": "male"
    }
}

response = requests.post(url, headers=headers, json=payload)
print(response.status_code)
print(response.json())


403
{'message': 'Access denied. Could not find a valid organization.'}
